In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#read csv
data = pd.read_csv('MY2010-2014 Fuel Consumption Ratings 5-cycle.csv', encoding='latin1', low_memory=False)
#copy daata
df = data.copy()
df

In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
df

### observation
- Empty columns from coloumn 15
- row 0 is part of the coloumn name
-

### Data Cleaning and Exploration

In [ ]:
for i, v in enumerate(df.columns):
    print(i, v)

In [ ]:
#drop multiple columns by slicing
df.drop(df.columns[13:], axis=1, inplace=True)

#reference : https://www.geeksforgeeks.org/how-to-drop-one-or-multiple-columns-in-pandas-dataframe/

In [ ]:
for i, v in enumerate(df.columns):
    print(i, v)

In [ ]:
df

In [ ]:
#rename variables
df = df.rename(columns={"MODEL":"MODEL YEAR","MODEL.1":"MODEL", "ENGINE SIZE":"ENGINE SIZE(L)","FUEL":"FUEL TYPE", "FUEL CONSUMPTION*":"FC CITY (L/100 km)",
                        "Unnamed: 9":"FC HWY (L/100 km)", "Unnamed: 10": "FC COMBINED(L/100 km)",
                        "Unnamed: 11":"FC COMBINED(mpg)","CO2 EMISSIONS ":"CO2 EMISSIONS"})

In [ ]:
df.head()

In [ ]:
#remove row 0
df = df.drop([0])
df.head()

In [ ]:
#reset index
df.reset_index(drop=True, inplace= True)

In [ ]:
df.head()

In [ ]:
df.info()

### observations
- variables that are meant to be float is showing as object

In [ ]:
#check for duplicated
df.duplicated().sum()

In [ ]:
#remove duplicates
df = df.drop_duplicates()

#reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()
#all coloumns have missing values

In [ ]:
df["MODEL YEAR"].unique()
#error

In [ ]:
df.tail(40)

In [ ]:
#issues from row 5360
#drop all rows from 5369
df.drop(df.index[5359:], inplace=True)

#reference: https://stackoverflow.com/questions/14661701/how-to-drop-a-list-of-rows-from-pandas-dataframe

In [ ]:
df.isna().sum()

In [ ]:
df["MODEL YEAR"].unique()

In [ ]:
df["MAKE"].unique()

In [ ]:
df["MODEL"].unique()

In [ ]:
df["VEHICLE CLASS"].unique()

In [ ]:
df["ENGINE SIZE(L)"].unique()

In [ ]:
#convert to float
df["ENGINE SIZE(L)"]= df["ENGINE SIZE(L)"].astype('float')
df["ENGINE SIZE(L)"].unique()

In [ ]:
df["CYLINDERS"].unique()

In [ ]:
df["TRANSMISSION"].unique()

In [ ]:
#df= df.replace({'TRANSMISSION':{'AS.*' : 'AS', 'AM.*' : 'AM', 'AV.*':'AV','M5':'M','M6':'M','M7':'M', 'A6':'A', 'A8':'A', 'A7':'A', 'A4':'A', 'A5':'A', 'A9':'A'} }, regex = True)                                                                                         

In [ ]:
#df["TRANSMISSION"].unique()

In [ ]:
df["FUEL TYPE"].unique()

In [ ]:
df["FC CITY (L/100 km)"].unique()

In [ ]:
#convert to float
df["FC CITY (L/100 km)"]= df["FC CITY (L/100 km)"].astype('float')
df["FC CITY (L/100 km)"].unique()

In [ ]:
df["FC HWY (L/100 km)"].unique()

In [ ]:
#convert to float
df["FC HWY (L/100 km)"]= df["FC HWY (L/100 km)"].astype('float')
df["FC HWY (L/100 km)"].unique()

In [ ]:
df["FC COMBINED(L/100 km)"].unique()

In [ ]:
#convert to float
df["FC COMBINED(L/100 km)"]= df["FC COMBINED(L/100 km)"].astype('float')
df["FC COMBINED(L/100 km)"].unique()

In [ ]:
df["FC COMBINED(mpg)"].unique()

In [ ]:
df["FC COMBINED(mpg)"]= df["FC COMBINED(mpg)"].astype('int')
df["FC COMBINED(mpg)"].unique()

In [ ]:
df["CO2 EMISSIONS"].unique()

In [ ]:
df["CO2 EMISSIONS"]= df["CO2 EMISSIONS"].astype('int')
df["CO2 EMISSIONS"].unique()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

### Univariate Analysis

In [ ]:
#check distribution
plt.figure(dpi=200);
df.hist(figsize = (20,15));
plt.show();

#### Observations
The distribution for CYLINDERS appears to be descrete.
The distribution for FUELCONSUMPTION_CITY, FUELCONSUMPTION_HWY and FUELCONSUMPTION_COMB appears to be right skewed. This can also be confirmed by looking at the mean
and median.

In [ ]:
#check fo ouliers
df.boxplot(figsize = (15,10), fontsize = 15, rot = 30)

Observations
- There are outliers in the data that needs to be treated
- Some variable exerts dominace over others and this will need to be standardixe or normalized

In [ ]:
def treat_outliers(df, col):
    """
    treats outliers in a varaible
    col: str, name of the numerical varaible
    df: data frame
    col: name of the column
    """
    Q1 = df[col].quantile(0.25)  # 25th quantile
    Q3 = df[col].quantile(0.75)  # 75th quantile
    IQR = Q3 - Q1
    Lower_Whisker = Q1 - 1.5 * IQR
    Upper_Whisker = Q3 + 1.5 * IQR
    df[col] = np.clip(
        df[col], Lower_Whisker, Upper_Whisker
    )  # all the values samller than Lower_Whisker will be assigned value of Lower_whisker
    # and all the values above upper_whisker will be assigned value of upper_Whisker
    return df


def treat_outliers_all(df, col_list):
    """
    treat outlier in all numerical varaibles
    col_list: list of numerical varaibles
    df: data frame
    """
    for c in col_list:
        df = treat_outliers(df, c)

    return df

### UNIVARIATE ANALYSIS

In [ ]:
# defining the dependent and independent variables
df_num = df.select_dtypes(exclude ='object') # obtains the numerical data ty
df_num = treat_outliers_all(df_num, df_num)

In [ ]:
object_df = df.select_dtypes(include ='object')

In [ ]:
#checking if outliers has ben treated
df_num.boxplot(figsize = (15,10), fontsize = 15, rot = 30)

In [ ]:
#checking for coreelation
plt.figure(figsize=(10,5))
corr_mat = df.corr()
sns.heatmap(data=corr_mat, annot = True, vmin=-1, cmap = "PiYG");

In [ ]:
sns.pairplot(data=df, diag_kind='kde')

 ### Linear Model Building

- We want to predict the C02 Emission.

- We'll be using all numerical continous variable

- We'll split the data into train and test to be able to evaluate the model that we build on the train data.

- We will build a Linear Regression model using the train data and then check it's performance.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_num.drop(['CO2 EMISSIONS'], axis= 1)
Y = df_num['CO2 EMISSIONS']

In [ ]:
#Split into train, test and validation()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1) #traintest split
X,Y=X_train,Y_train

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=1) #train validation split

In [ ]:
print("Number of rows in test data =", X_test.shape[0])
print("Number of rows in train data =", X_train.shape[0])
print("Number of rows in validation data =", X_val.shape[0])

In [ ]:
len(df)

### Scaling the data
- The two most common scaller is minmax scaler and  standard scaler
- since this is a linear regression problem. Standardiztion will be the best option

- Normalization is useful when your data has varying scales and the algorithm you are using does not make assumptions about the distribution of your data, such as k-nearest neighbors and artificial neural networks.

- Standardization is useful when your data has varying scales and the algorithm you are using does make assumptions about your data having a Gaussian distribution, such as linear regression, logistic regression, and linear discriminant analysis

#ref:https://towardsai.net/p/data-science/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff#:~:text=Normalization%20is%20useful%20when%20your,Gaussian%20(bell%20curve)%20distribution.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#standardize
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [ ]:
#fit training data
from sklearn.linear_model import LinearRegression
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_scaled, Y_train)

### Check Model Performance

In [ ]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# # function to compute MAPE
# def mape_score(targets, predictions):
#     return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
#     mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2
        },
        index=[0],
    )

    return df_perf

In [ ]:
# to check model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
lin_reg_model_perf_train = model_performance_regression(lin_reg_model, X_train_scaled, Y_train)
lin_reg_model_perf_train

In [ ]:
# Checking model performance on test set
lin_reg_model_perf_test = model_performance_regression(lin_reg_model, X_test_scaled, Y_test)
lin_reg_model_perf_test

In [ ]:
lin_reg_model_perf_val = model_performance_regression(lin_reg_model, X_val_scaled, Y_val)
lin_reg_model_perf_val

In [ ]:
print('Training performance')
print(lin_reg_model_perf_train)
print('')
print('Test performance')
print(lin_reg_model_perf_test)
print('')
print('validation performance')
print(lin_reg_model_perf_val)

#### Performance Improvement

The model underfits i.e not performing well

There major reason for this might be because the model isnt complex enough

will be adding more variables to make the model more complex

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
ohe = OneHotEncoder() # declares an instance of the object
ohe_dat=ohe.fit_transform(object_df[['FUEL TYPE']]).toarray() # applies the object to data
feature_labels = ohe.categories_ # the labels are stored here
feature_labels = np.array(feature_labels).ravel()
df_ohe = pd.DataFrame(ohe_dat, columns = feature_labels ) # creating a dataframe for the ohe variable
df_ohe_new = pd.concat([df_num,df_ohe], axis = 1) # combining the original dataframe (df) and df_ohe
df_ohe_new

In [ ]:
plt.figure(figsize=(10,5))
corr_mat = df_ohe_new.corr()
sns.heatmap(data=corr_mat, annot = True, vmin=-1, cmap = "PiYG");
#plt.subplots_adjust(right=0.8, left=0.2)
plt.savefig('Correlation Fuel type')

In [ ]:
df2 = df_ohe_new.drop(['D'], axis = 1)
#very low correclation

In [ ]:
X2 = df2.drop(['CO2 EMISSIONS'], axis= 1)
Y2 = df2['CO2 EMISSIONS']

In [ ]:
#Split into train, test and validation()

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.2, random_state=1) #traintest split
X2,Y2=X2_train,Y2_train

X2_train, X2_val, Y2_train, Y2_val = train_test_split(X2, Y2, test_size=0.2, random_state=1) #train validation split

In [ ]:
#fit training data
#standardize
scaler = StandardScaler()

X2_train_scaled = scaler.fit_transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)
X2_val_scaled = scaler.transform(X2_val)

In [ ]:
lin_reg_model2 = LinearRegression()
lin_reg_model2.fit(X2_train_scaled, Y2_train)

In [ ]:
lin_reg_model_perf_train = model_performance_regression(lin_reg_model2, X2_train_scaled, Y2_train)
lin_reg_model_perf_train

In [ ]:
lin_reg_model_perf_test = model_performance_regression(lin_reg_model2, X2_test_scaled, Y2_test)
lin_reg_model_perf_test

In [ ]:
lin_reg_model_perf_val = model_performance_regression(lin_reg_model2, X2_val_scaled, Y2_val)
lin_reg_model_perf_val

In [ ]:
print('Training performance')
print(lin_reg_model_perf_train)
print('')
print('Test performance')
print(lin_reg_model_perf_test)
print('')
print('validation performance')
print(lin_reg_model_perf_val)

#### Question 3

In [ ]:
#df['MODEL YEAR']= df['MODEL YEAR'].astype('int')

In [ ]:
sns.lineplot(data=df, x ='MODEL YEAR', y='CO2 EMISSIONS');
plt.title('CO2 Emission by Year');
plt.savefig('Yearly CO2 Emissions')

#### There's constant reduction in fuel consumption from 2012 to 2014

#### Question 4

#### Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, KFold, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC # SVC: support vector classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (accuracy_score,precision_score, recall_score,f1_score,precision_recall_curve,
                             classification_report,confusion_matrix, ConfusionMatrixDisplay, roc_curve,roc_auc_score)

In [ ]:
# Appending several models for cross validation
models = []
models.append(("LR",LogisticRegression()))
models.append(("KNN",KNeighborsClassifier()))
models.append(("Dtree",DecisionTreeClassifier()))
models.append(("SVC",SVC()))
models.append(("RF",RandomForestClassifier()))
models

In [ ]:
def report_cross_val_score(models):
    for name,model in models:
        kfold = KFold(n_splits=5)
        cv_result = cross_val_score(model,X_train,Y_train, cv = kfold,scoring = "accuracy")
        #print(name, cv_result.mean())
        print(name, cv_result)


report_cross_val_score(models)

In [ ]:
# The model test report
def report_model_test(models):
    for name,model in models:
        model.fit(X_train_scaled,Y_train)
        basem_preds = model.predict(X_test_scaled)
        print(name)
        print(classification_report(Y_test,basem_preds))
        ConfusionMatrixDisplay.from_estimator(model, X_test_scaled, Y_test)
        plt.show()
        plt.figure(figsize=(40,40), dpi = 200);


report_model_test(models)

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

In [ ]:
le = LabelEncoder()# declares an instance of the object
le_data = object_df.apply(le.fit_transform)# applies the object to data
df_le = pd.DataFrame(le_data) # creates a dataframe
new_df_le = pd.concat([df_num,df_le], axis = 1)
new_df_le

In [ ]:
plt.figure(figsize=(15,10))
corr_mat = new_df_le.corr()
plt.subplots_adjust(bottom = 0.3)
sns.heatmap(data=corr_mat, annot = True, vmin=-1, cmap = "PiYG");
plt.savefig(' All Corr Matt')

In [ ]:
new_df_le =new_df_le.drop(['MODEL YEAR', 'MODEL'], axis = 1)

In [ ]:
plt.figure(figsize=(15,10))
corr_mat = new_df_le.corr()
sns.heatmap(data=corr_mat, annot = True, vmin=-1, cmap = "PiYG");

In [ ]:
def report_model(model, predictor, target):
    preds = model.predict(predictor)

    feature_importance=pd.DataFrame(data=model.feature_importances_, index =predictor.columns,
    columns = ['Feature_importance']).sort_values('Feature_importance')
    print(feature_importance)
    print(classification_report(target,preds))
    plt.rcParams["figure.figsize"] = (15,10)
    ConfusionMatrixDisplay.from_estimator(model,predictor, target)

    plt.show()
    plot_tree(model, feature_names = predictor.columns, filled = True, fontsize=20 );

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
fig, ax_position=plt.subplots(1,2,figsize=(15,6),dpi=200) # creates the framework for the plotting
a = sns.countplot(x='MAKE', data=new_df_le, ax=ax_position[0]) #ax_position[0] specifies plot to be in index 0
a= df['MAKE'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax_position[1]) #ax_position[1] specifies plot to be in index 1


In [ ]:
#using 'MAKE' as target
x = new_df_le.drop(['MAKE','VEHICLE CLASS','FUEL TYPE'], axis=1)
y = new_df_le['MAKE']

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
from imblearn.over_sampling import SMOTE
sm = RandomOverSampler(random_state=42) # The object is created
x_res, y_res = sm.fit_resample(x, y) # The object is applied
x, y = x_res, y_res # reassigning the balanced dataset to X,y

In [ ]:
# Plot of the dataset
df_bal = pd.concat([x_res,y_res], axis = 1) # creating a dataframe for the balanced data
fig, ax=plt.subplots(1,2,figsize=(15,6)) # creating the axis shell for subplot
a = sns.countplot(x='MAKE',data=df_bal, ax=ax[0]) # assigning each of the plot to the axis shell
a= df_bal['MAKE'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax[1]) # assigning each of the plot to the axis shell

In [ ]:
#Split into train, test and validation()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1) #traintest split
x,y=x_train,y_train

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1) #train validation split

In [ ]:
model.fit(x_train, y_train)

In [ ]:
report_model(model, x_train, y_train)

In [ ]:
report_model(model, x_test, y_test)

In [ ]:
report_model(model, x_val, y_val)

In [ ]:
fig, ax_position=plt.subplots(1,2,figsize=(15,6),dpi=200) # creates the framework for the plotting
a = sns.countplot(x='VEHICLE CLASS', data=new_df_le, ax=ax_position[0]) #ax_position[0] specifies plot to be in index 0
a= df['VEHICLE CLASS'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax_position[1]) #ax_position[1] specifies plot to be in index 1

In [ ]:
#using VEHICLE CLASS as target
x2 = new_df_le.drop(['TRANSMISSION', 'CYLINDERS', 'ENGINE SIZE(L)', 'VEHICLE CLASS', 'FUEL TYPE', 'MAKE'], axis=1)
y2= new_df_le['VEHICLE CLASS']

In [ ]:
sm = RandomOverSampler(random_state=42) # The object is created
x2_res, y2_res = sm.fit_resample(x2, y2) # The object is applied
x2, y2 = x2_res, y2_res # reassigning the balanced dataset to X,y

In [ ]:
# Plot of the dataset
df_bal = pd.concat([x2_res,y2_res], axis = 1) # creating a dataframe for the balanced data
fig, ax=plt.subplots(1,2,figsize=(15,6)) # creating the axis shell for subplot
a = sns.countplot(x='VEHICLE CLASS',data=df_bal, ax=ax[0]) # assigning each of the plot to the axis shell
a= df_bal['VEHICLE CLASS'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax[1]) # assigning each of the plot to the axis shell

In [ ]:
#Split into train, test and validation()
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=1) #traintest split
x2,y2=x2_train,y2_train

x2_train, x2_val, y2_train, y2_val = train_test_split(x2, y2, test_size=0.2, random_state=1) #train validation split

In [ ]:
model.fit(x2_train, y2_train)

In [ ]:
report_model(model, x2_train, y2_train)

In [ ]:
report_model(model, x2_test, y2_test)

In [ ]:
report_model(model, x2_val, y2_val)

In [ ]:
fig, ax_position=plt.subplots(1,2,figsize=(15,6),dpi=200) # creates the framework for the plotting
a = sns.countplot(x='TRANSMISSION', data=new_df_le, ax=ax_position[0]) #ax_position[0] specifies plot to be in index 0
a= df['TRANSMISSION'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax_position[1]) #ax_position[1] specifies plot to be in index 1

In [ ]:
#using TRANSMISION as taerget
x3 = new_df_le.drop(['TRANSMISSION', 'VEHICLE CLASS'], axis=1)
y3= new_df_le['TRANSMISSION']

In [ ]:
sm = RandomOverSampler(random_state=42) # The object is created
x3_res, y3_res = sm.fit_resample(x3, y3) # The object is applied
x3, y3 = x3_res, y3_res # reassigning the balanced dataset to X,y

In [ ]:
# Plot of the dataset
df_bal = pd.concat([x3_res,y3_res], axis = 1) # creating a dataframe for the balanced data
fig, ax=plt.subplots(1,2,figsize=(15,6)) # creating the axis shell for subplot
a = sns.countplot(x='TRANSMISSION',data=df_bal, ax=ax[0]) # assigning each of the plot to the axis shell
a= df_bal['TRANSMISSION'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax[1]) # assigning each of the plot to the axis shell

In [ ]:
#Split into train, test and validation()
x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.2, random_state=1) #traintest split
x3,y3=x3_train,y3_train

x3_train, x3_val, y3_train, y3_val = train_test_split(x3, y3, test_size=0.2, random_state=1) #train validation split

In [ ]:
model.fit(x3_train, y3_train)

In [ ]:
report_model(model, x3_train, y3_train)

In [ ]:
report_model(model, x3_test, y3_test)

In [ ]:
report_model(model, x3_val, y3_val)

In [ ]:
fig, ax_position=plt.subplots(1,2,figsize=(15,6),dpi=200) # creates the framework for the plotting
a = sns.countplot(x='FUEL TYPE', data=new_df_le, ax=ax_position[0]) #ax_position[0] specifies plot to be in index 0
a= df['FUEL TYPE'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax_position[1]) #ax_position[1] specifies plot to be in index 1

In [ ]:
#using FUEL TYPE as target
x4 = new_df_le[['TRANSMISSION', 'FC HWY (L/100 km)']]
y4= new_df_le['FUEL TYPE']

In [ ]:
from imblearn.over_sampling import SMOTE
sm = RandomOverSampler(random_state=42) # The object is created
x4_res, y4_res = sm.fit_resample(x4, y4) # The object is applied
x4, y4 = x4_res, y4_res # reassigning the balanced dataset to X,y

In [ ]:
# Plot of the dataset
df_bal = pd.concat([x4_res,y4_res], axis = 1) # creating a dataframe for the balanced data
fig, ax=plt.subplots(1,2,figsize=(15,6)) # creating the axis shell for subplot
a = sns.countplot(x='FUEL TYPE',data=df_bal, ax=ax[0]) # assigning each of the plot to the axis shell
a= df_bal['FUEL TYPE'].value_counts().plot.pie(autopct="%1.1f%%", ax=ax[1]) # assigning each of the plot to the axis shell

In [ ]:

#Split into train, test and validation()
x4_train, x4_test, y4_train, y4_test = train_test_split(x4, y4, test_size=0.2, random_state=1) #traintest split
x4,y4=x4_train,y4_train

x4_train, x4_val, y4_train, y4_val = train_test_split(x4, y4, test_size=0.2, random_state=1) #train validation split

In [ ]:
model.fit(x4_train, y4_train)

In [ ]:
report_model(model, x4_train, y4_train)

In [ ]:
report_model(model, x4_test, y4_test)

In [ ]:
report_model(model, x4_val, y4_val)

#### Clustering

In [ ]:
# ploting the attributes to see how they affect the clusters formed
for col1 in df_num.columns:
    for col2 in df_num.columns:
        plt.figure(figsize = (8,9),dpi = 200)
        sns.set(font_scale=2)
        sns.scatterplot(data = df_num, x = df_num[col1],y = df_num[col2],
        hue='CYLINDERS', palette = 'viridis')
        plt.show()

In [ ]:
# Plotting the histogram to see their relationship with categories
for col in df_num.columns:
    sns.set(font_scale=2)
    plt.figure(figsize=(20,15), dpi = 200)
    sns.histplot(data=df,x = col, hue = 'CYLINDERS', palette = 'viridis')
    plt.xticks(rotation=90)

In [ ]:
# Plotting the count of the values to see the useful categories
for col in object_df.columns:
    sns.set(font_scale=2)
    plt.figure(figsize=(20,15), dpi = 200)
    sns.countplot(data = object_df,x=col, order = object_df[col].value_counts().index)
    plt.xticks(rotation = 90)

In [ ]:
#Encodeded caterogical data
df_le

In [ ]:
#drop model year and model as it has little effect to the data
obj_df = df_le.drop(['MODEL YEAR'], axis =1)

In [ ]:
obj_df

#### Scaling the dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaled_num_df=scaler.fit_transform(df_num)
scaled_num_df

In [ ]:
scaled_obj_df=scaler.fit_transform(obj_df)
scaled_obj_df

#### Kmeans Clusttering

In [ ]:
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(scaled_num_df)
    Sum_of_squared_distances.append(km.inertia_)
#Visualing the plot
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.savefig('Elbow Curve')
plt.show()

#### Elbow curve is at 2 which means optimal value of k is 2

#### Training the Model

In [ ]:
from sklearn.cluster import KMeans # import KMeans 

In [ ]:
model = KMeans(n_clusters=2, random_state = 0) # assigning the number of clusters to 2

In [ ]:
cluster_labels = model.fit_predict(scaled_num_df) # fiting and predicting the clusters using numerical variabies
cluster_labels

In [ ]:
obj_df['CLUSTER']=cluster_labels

In [ ]:
obj_df

In [ ]:
obj_df.drop('CLUSTER', axis = 1, inplace = True)
for col1 in obj_df.columns:
    for col2 in obj_df.columns:
        plt.figure(figsize = (8,9),dpi = 200)
        sns.set(font_scale=2)
        sns.scatterplot(data = obj_df, x = obj_df[col1],y = obj_df[col2],
        hue=cluster_labels, palette = 'viridis')
        plt.savefig('all clus')
        plt.show()

### Clustering evaluation

In [ ]:
#### Internal Evaluation
from sklearn.metrics import davies_bouldin_score, silhouette_score

In [ ]:
DB_index=davies_bouldin_score(scaled_obj_df,cluster_labels) #df_le = Encoded Cateogircal varialbes
DB_index                                            #cluster_label = only numerical variables used

In [ ]:
sil_score=silhouette_score(scaled_obj_df,cluster_labels)
sil_score

In [ ]:
from sklearn.cluster import AgglomerativeClustering #here we import

In [ ]:
model = AgglomerativeClustering(n_clusters=7,linkage='single')

In [ ]:
cluster_labels=model.fit_predict(scaled_num_df)
cluster_labels

In [ ]:
DB_index=davies_bouldin_score(scaled_obj_df,cluster_labels)
DB_index

In [ ]:
sil_score=silhouette_score(scaled_obj_df,cluster_labels)
sil_score

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
model = AgglomerativeClustering(n_clusters=None, distance_threshold=0) # distance_threshold=0 means all points

In [ ]:
cluster_labels=model.fit_predict(scaled_num_df)
cluster_labels

In [ ]:
from scipy.cluster import hierarchy # scipy is used for plotting the hierarchy
from scipy.cluster.hierarchy import dendrogram

In [ ]:
matrix_linkage = hierarchy.linkage(model.children_) # hierarchy is used to obtain the children
matrix_linkage # distances between the points 

In [ ]:
plt.figure(figsize=(20,12), dpi = 200,)
dendrogram(Z=matrix_linkage,truncate_mode= 'level', p= 7);